# Surprisal 실습

## 초기 설정

아래 셀을 실행하여 초기 설정을 수행해 주세요.

In [ ]:
!bash /content/setup.sh

In [2]:
import torch
import sys

import pandas as pd

from sys import platform
from os import path
from torch import device
from transformers import BertTokenizer, BertForMaskedLM
from tqdm import tqdm
from surprisal import bert_token_surprisal

In [ ]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

## 모델 불러오기

### 영어

BERT(large, uncased)를 불러옵니다.

In [ ]:
model_name_eng = 'bert-large-uncased'
tokenizer_eng = BertTokenizer.from_pretrained(model_name_eng)
model_eng = BertForMaskedLM.from_pretrained(model_name_eng, output_attentions=True)

### 한국어
KR-BERT를 불러옵니다.

In [ ]:
model_name_kr= "snunlp/KR-BERT-char16424"
tokenizer_kr = BertTokenizer.from_pretrained(model_name_kr, do_lower_case=False)
model_kr =BertForMaskedLM.from_pretrained(model_name_kr, output_attentions=True)

# (1차원) 요인설계실험

## 텍스트 설정

* 사용법
    * 공통되는 토큰을 [MASK]로 치환합니다.
    * 입력으로 들어가는 토큰을 키워드로 지정합니다.

In [ ]:
text = """
철수가 영희[MASK] 사랑한다."""

## Surprisal

In [ ]:
bert_token_surprisal(text, ["##을", "##를"], model_kr, tokenizer_kr, device)

## 영어 텍스트 설정

In [5]:
text = """
John [MASK] Mary"""

## Surprisal

In [ ]:
bert_token_surprisal(text, ["love", "loves"], model_eng, tokenizer_eng, device)

# (2차원) 요인설계실험

## 텍스트 설정

* 사용법
    * 공통되는 토큰을 [MASK]로 치환합니다.
    * 입력으로 들어가는 토큰을 키워드로 지정합니다.

In [ ]:
text = """
철수가 영희[MASK] 좋아한다.
철수는 영희[MASK] 좋아한다."""

## Surprisal

In [ ]:
bert_token_surprisal(text, ["##을", "##를"], model_kr, tokenizer_kr, device)

## 영어 텍스트 설정

In [11]:
text = """
I know [MASK] he ate yesterday.
I know [MASK] he ate pancakes yesterday"""

## Surprisal

In [ ]:
bert_token_surprisal(text, ["what", "that"], model_eng, tokenizer_eng, device)

# 여러 문장의의 Surprisal 확인하기

In [6]:
LANGUAGE = "Korean"

In [7]:
if LANGUAGE.lower() == "korean":
  model_name = model_kr
  tokenizer = tokenizer_kr

elif LANGUAGE.lower() == "english":
  model_name = model_eng
  tokenizer = tokenizer_eng

else:
  print("이번 튜토리얼에서는는 한국어와 영어어 데이터만을을 다룹니다.")

## 데이터 불러오기

In [12]:
filename = "/content/non_acc.xlsx"

output_name = "/" + path.splitext(filename)[0] + "_output.txt"
lines = pd.read_excel(filename, index_col="IDX")

## 계산 진행하기

In [ ]:
f = open(output_name, 'w')

f.write("IDX")
f.write("\tSEN")
for i in range(len(lines.columns)):
  if i == 0:
    continue
  elif i == len(lines.columns)-1:
    item = "\tITEM{}".format(i)
    f.write(item + "\n")
  else:
    item = "\tITEM{}".format(i)
    f.write(item)

for index, line in tqdm(lines.iterrows()):
    each = line.to_list()
    text = each[0].strip()
    keywords = []
    for i in range(len(each)):
      if i == 0:
        continue
      keyword = each[i].strip()
      keywords.append(keyword)
      
    result = bert_token_surprisal(text, keywords,  model_name, tokenizer, device, printing=False)
    scores = ""
    
    for res in result:
      scores += str(res[2])
      scores += "\t"    
    f.write(str(index) + "\t" + each[0] + "\t" + scores.strip() + "\n")
  
f.close()

## 결과 파일을 엑셀로 변환하기

In [14]:
df = pd.read_csv(output_name, sep='\t', index_col=0, header=0)

In [15]:
excel_name = path.splitext(output_name)[0] + '.xlsx'
df.to_excel(excel_name)